In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'

In [2]:
import json
from tqdm.auto import tqdm

In [3]:
import sys
sys.path.append('/home/majed_alshaibani/Projects/jrcai_corekit/src')

In [4]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) # suppress requests warnings of InsecureRequestWarning

In [5]:
from llm import *


/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [6]:
llm_loader = LLMLoader(
    '/hdd/shared_models/Meta-Llama-3.1-70B-Instruct/',
    llm_initializer=Llama31Initializer(),
    )
messsage_generator = MessageGeneratorFromLocalLLM(llm_loader)

# Exploring Llama outputs

In [7]:
def get_chat_completion(messages):
    llm_response = messsage_generator(messages)
    return llm_response[0]

test the API

In [8]:
print(get_chat_completion(
    messages=[[{
            'role': 'user',
            'content': 'ما هو لون نجوم السماء؟'
        }]]
    )
)

loading configuration file /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/config.json
Model config LlamaConfig {
  "_name_or_path": "/hdd/shared_models/Meta-Llama-3.1-70B-Instruct/",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_size": 28672,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 64,
  "num_hidden_layers": 80,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.43.3",
  "use_cac

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
loading configuration file /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}

loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/generation_config.json
Generate 

Meta-Llama-3.1-70B-Instruct - batches sizes = [1]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 1/1 [00:17<00:00, 17.05s/it]

لون النجوم يختلف بناءً على درجة حرارة سطحها. تُصنف النجوم إلى عدة فئات بناءً على درجات حرارتها السطحية، وبالتالي تختلف ألوانها. إليك بعض الألوان الشائعة للنجوم:
نجوم حارة: تظهر باللون الأبيض أو الأزرق، مثل سيرتيس في كوكبة القيثارة.
نجوم معتدلة: تظهر باللون الأبيض أو الأبيض المصفر، مثل الشمس.
نجوم باردة: تظهر باللون البرتقالي أو الأحمر، مثل بيتا القيثارة.


# Generating abstracts

## utilities functions

In [9]:
def save_abstracts_to_jsonl(articles, file_path):  
    # Append new articles
    with open(file_path, 'w', encoding='utf-8') as f:
        for article in articles:
            json_object = article
            f.write(json.dumps(json_object, ensure_ascii=False) + '\n')

def load_existing_abstracts(file_path):
    """Load existing abstracts from a JSONL file, if it exists."""
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as file:
            return [json.loads(line) for line in file]
    return []

In [10]:
def load_abstracts():
    abstracts = []
    with open('arabic_datasets/arabic_filtered_papers.json', 'r', encoding='utf-8') as f:
        abstracts = json.load(f)
    return abstracts

In [11]:
abstracts = load_abstracts()
len(abstracts)

3000

In [12]:
def generate_abstracts(generation_prompts, generation_type, batch_size=100):
    # Define file path for saving abstracts
    os.makedirs("generated_arabic_datasets/llama-batched/arabic_abstracts_dataset", exist_ok=True)
    file_path = f"generated_arabic_datasets/llama-batched/arabic_abstracts_dataset/{generation_type}_abstracts_generation.jsonl"
    
    # Load already generated abstracts to resume if script is interrupted
    generated_abstracts = load_existing_abstracts(file_path)
    start_idx = len(generated_abstracts)
    print(f"Resuming from batch {start_idx // batch_size}...")

    # Prepare batches of prompts starting from the last completed batch
    batched_messages = [
        [
            [{'role': 'user', 'content': prompt}] 
            for prompt in generation_prompts[i:i+batch_size]
        ]
        for i in range(0, len(generation_prompts), batch_size)
    ]
    
    # Main process
    for batch_idx, batch in enumerate(tqdm(batched_messages[start_idx // batch_size:], desc="Processing batches"), start=start_idx // batch_size):
        generated_batch = messsage_generator(batch)
        batch_generated_abstracts = []
        
        for abstract in generated_batch:
            abstract = abstract.strip().strip('<END>').strip().strip('<START>').strip()
            end_token_index = abstract.find('<END>')
            if end_token_index > 0:
                abstract = abstract[:end_token_index]
            batch_generated_abstracts.append(abstract.strip())
        
        # batch_generated_abstracts = generated_batch

        # Get the current slice of the abstracts
        batch_start = batch_idx * batch_size
        batch_end = min((batch_idx + 1) * batch_size, len(generation_prompts))
        
        # Format the results for this batch
        batch_results = [
            {
                "original_abstract": abstract['arabic_abstract'],
                "generated_abstract": generated_abstract,
            }
            for abstract, generated_abstract in zip(
                abstracts[batch_start:batch_end],
                batch_generated_abstracts
            )
        ]

        # Extend the accumulated results and save after each batch
        generated_abstracts.extend(batch_results)
        save_abstracts_to_jsonl(generated_abstracts, file_path)

    print(f"All batches processed and saved to {file_path}")

## Generate from title only

In [13]:
generate_from_title_prompt = """
أنت كاتب أكاديمي خبير متخصص في كتابة الملخصات البحثية. مهمتك الأساسية هي إنشاء ملخص موجز ومعبر من فقرة واحدة للورقة البحثية التي عنوانها سيُعطى لك.

القواعد الأساسية:
1. يجب أن يتراوح الملخص بين 100-150 كلمة بالضبط
2. يجب أن يكون الملخص شاملاً ويغطي: الهدف الرئيسي، المنهجية، والنتائج المتوقعة
3. لا يُسمح بإنشاء ملخص فارغ تحت أي ظرف
4. استخدم أسلوباً علمياً رصيناً يناسب الأوراق البحثية

التنسيق المطلوب:
- ابدأ الملخص مباشرة بعد علامة <START>
- انتهِ بعلامة <END>
- لا تضف أي نص خارج هذه العلامات

العنوان:
{title}

الملخص:
<START>
""".strip()

In [14]:
print(generate_from_title_prompt.format(title=abstracts[0]['title']))

أنت كاتب أكاديمي خبير متخصص في كتابة الملخصات البحثية. مهمتك الأساسية هي إنشاء ملخص موجز ومعبر من فقرة واحدة للورقة البحثية التي عنوانها سيُعطى لك.

القواعد الأساسية:
1. يجب أن يتراوح الملخص بين 100-150 كلمة بالضبط
2. يجب أن يكون الملخص شاملاً ويغطي: الهدف الرئيسي، المنهجية، والنتائج المتوقعة
3. لا يُسمح بإنشاء ملخص فارغ تحت أي ظرف
4. استخدم أسلوباً علمياً رصيناً يناسب الأوراق البحثية

التنسيق المطلوب:
- ابدأ الملخص مباشرة بعد علامة <START>
- انتهِ بعلامة <END>
- لا تضف أي نص خارج هذه العلامات

العنوان:
صور عن نظام التعليم عند المرأة الأندلسية

الملخص:
<START>


In [15]:
generate_from_title_prompts = []
for abstract in abstracts:
    prompt = generate_from_title_prompt.format(title=abstract['title'])
    generate_from_title_prompts.append(prompt)

In [16]:
generate_from_title_prompts[:3]

['أنت كاتب أكاديمي خبير متخصص في كتابة الملخصات البحثية. مهمتك الأساسية هي إنشاء ملخص موجز ومعبر من فقرة واحدة للورقة البحثية التي عنوانها سيُعطى لك.\n\nالقواعد الأساسية:\n1. يجب أن يتراوح الملخص بين 100-150 كلمة بالضبط\n2. يجب أن يكون الملخص شاملاً ويغطي: الهدف الرئيسي، المنهجية، والنتائج المتوقعة\n3. لا يُسمح بإنشاء ملخص فارغ تحت أي ظرف\n4. استخدم أسلوباً علمياً رصيناً يناسب الأوراق البحثية\n\nالتنسيق المطلوب:\n- ابدأ الملخص مباشرة بعد علامة <START>\n- انتهِ بعلامة <END>\n- لا تضف أي نص خارج هذه العلامات\n\nالعنوان:\nصور عن نظام التعليم عند المرأة الأندلسية\n\nالملخص:\n<START>',
 'أنت كاتب أكاديمي خبير متخصص في كتابة الملخصات البحثية. مهمتك الأساسية هي إنشاء ملخص موجز ومعبر من فقرة واحدة للورقة البحثية التي عنوانها سيُعطى لك.\n\nالقواعد الأساسية:\n1. يجب أن يتراوح الملخص بين 100-150 كلمة بالضبط\n2. يجب أن يكون الملخص شاملاً ويغطي: الهدف الرئيسي، المنهجية، والنتائج المتوقعة\n3. لا يُسمح بإنشاء ملخص فارغ تحت أي ظرف\n4. استخدم أسلوباً علمياً رصيناً يناسب الأوراق البحثية\n\nالتنسيق المطل

In [17]:
generate_abstracts(generation_prompts=generate_from_title_prompts, generation_type='from_title')

Resuming from batch 0...


Processing batches:   0%|          | 0/30 [00:00<?, ?it/s]

Meta-Llama-3.1-70B-Instruct - batches sizes = [13, 15, 15, 16, 16, 16, 9]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 7/7 [06:18<00:00, 54.01s/it]


Meta-Llama-3.1-70B-Instruct - batches sizes = [14, 15, 15, 15, 16, 16, 9]


...

Meta-Llama-3.1-70B-Instruct - batches sizes = [14, 15, 15, 15, 16, 16, 9]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 7/7 [05:32<00:00, 47.48s/it]


Meta-Llama-3.1-70B-Instruct - batches sizes = [15, 15, 15, 16, 16, 16, 7]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 7/7 [05:06<00:00, 43.77s/it]

All batches processed and saved to generated_arabic_datasets/llama-batched/arabic_abstracts_dataset/from_title_abstracts_generation.jsonl


## Generate from title and content

In [13]:
max_paper_content_length = 400

In [14]:
generate_from_title_and_content_prompt = """
أنت مساعد أكاديمي خبير متخصص في تحليل وإعادة صياغة النصوص العربية. مهمتك هي تحليل العنوان والمحتوى المقدم لك (بحد أقصى {max_paper_content_length} كلمة) وإنشاء ملخص موجز وفعال من 100-150 كلمة.

مهمتك الأساسية هي إنشاء ملخص أكاديمي (100-150 كلمة) في جميع الحالات، حتى لو كان النص المقدم غير مفهوم تماماً.

القاعدة الذهبية: يجب عليك دائماً إنشاء ملخص أكاديمي محترف، حتى لو كان عليك الاعتماد على العنوان فقط.

ملاحظات مهمة:
المحتوى المقدم مستخرج من ملف PDF وقد يحتوي على أخطاء كثيرة في التحويل
قد تجد بعض المشاكل الشائعة مثل:
- تداخل الحروف العربية بشكل كبير
- تكرار بعض الحروف بشكل غير منطقي (مثل: السسسلام)
- تشويه كبير في بعض الكلمات لدرجة قد تجعلها غير مفهومة
- فصل غير صحيح للكلمات
- أخطاء في علامات التشكيل
- تحويل خاطئ لبعض الحروف المتشابهة مثل (ا/أ/إ) و (ه/ة)
- اختفاء بعض الحروف أو استبدالها برموز غير مفهومة

إرشادات مهمة:
- تجاهل تماماً أي ملخص أو مستخلص موجود في المحتوى المقدم (عادةً ما يكون في بداية البحث). لم نقم بحذفه لأنه من الصعب حذف الملخص الموجود مسبقاً من النص المدخل، إلا أن مهمتك هي تجاهله بشكل كامل وإنشاء ملخص جديد مستقل بناءً على محتوى البحث نفسه. لا تتأثر بالملخص الموجود ولا تعتمد عليه في صياغة الملخص الجديد.
- استخدام سياق النص لفهم المعنى المقصود وتصحيح الأخطاء ذهنياً.
- التركيز على الأفكار الواضحة والمفهومة في النص.
- في حال وجود أجزاء غير مفهومة بسبب التشويه، يمكنك الاعتماد على الأجزاء المفهومة.
- في حال كان النص غير مفهوم بشكل كبير، قم بإنشاء ملخص من عنوان البحث فقط، مع التركيز على الموضوع الرئيسي والمجال العلمي والأهداف المتوقعة.

المطلوب:
- قراءة العنوان والمحتوى بعناية.
- فهم الأفكار الرئيسية رغم أخطاء التحويل الكثيرة.
- إنشاء ملخص واضح ومتماسك يعكس جوهر البحث.
- الالتزام بالأسلوب الأكاديمي الرصين.

تنبيهات هامة للملخص:
- يجب أن يكون الملخص نصاً متصلاً بدون أسطر فارغة أو فواصل سطرية.
- لا تترك الملخص فارغاً تحت أي ظرف.
- يجب أن يكون الملخص أكاديمياً بحتاً، يصف محتوى البحث أو موضوعه وأهدافه.
- ممنوع منعاً باتاً كتابة عبارات مثل "النص غير واضح" أو "يصعب تقديم ملخص" أو "للأسف، النص المقدم غير واضح" أو أي تعليقات حول جودة النص، قم مباشرة بإنشاء ملخص أكاديمي من العنوان دون الإشارة إلى مشاكل النص.
- في حال عدم وضوح النص، قم مباشرة بإنشاء ملخص أكاديمي من العنوان دون الإشارة إلى مشاكل النص.
- لا تضف مسافات أو أسطر فارغة في بداية أو نهاية الملخص.
- اكتب الملخص مباشرة بعد علامة <START> بدون أي مسافات إضافية.

تذكر: مهمتك الأساسية هي إنتاج ملخص أكاديمي محترف في جميع الحالات. عدم فهم النص ليس عذراً لعدم إنتاج ملخص - في هذه الحالة، استخدم العنوان لإنشاء ملخص يتناول الموضوع من منظور أكاديمي عام.

يرجى البدء مباشرة بعد علامة <START> والانتهاء عند علامة <END>. لا تضف أي نص خارج هذه العلامات.

المحتوى:
{content}

العنوان:
{title}

الملخص:
<START>
""".strip()

In [15]:
print(generate_from_title_and_content_prompt.format(title=abstracts[0]['title'],content=' '.join(abstracts[0]['paper_extracted_content'].split(' ')[:max_paper_content_length]),max_paper_content_length=max_paper_content_length))

أنت مساعد أكاديمي خبير متخصص في تحليل وإعادة صياغة النصوص العربية. مهمتك هي تحليل العنوان والمحتوى المقدم لك (بحد أقصى 400 كلمة) وإنشاء ملخص موجز وفعال من 100-150 كلمة.

مهمتك الأساسية هي إنشاء ملخص أكاديمي (100-150 كلمة) في جميع الحالات، حتى لو كان النص المقدم غير مفهوم تماماً.

القاعدة الذهبية: يجب عليك دائماً إنشاء ملخص أكاديمي محترف، حتى لو كان عليك الاعتماد على العنوان فقط.

ملاحظات مهمة:
المحتوى المقدم مستخرج من ملف PDF وقد يحتوي على أخطاء كثيرة في التحويل
قد تجد بعض المشاكل الشائعة مثل:
- تداخل الحروف العربية بشكل كبير
- تكرار بعض الحروف بشكل غير منطقي (مثل: السسسلام)
- تشويه كبير في بعض الكلمات لدرجة قد تجعلها غير مفهومة
- فصل غير صحيح للكلمات
- أخطاء في علامات التشكيل
- تحويل خاطئ لبعض الحروف المتشابهة مثل (ا/أ/إ) و (ه/ة)
- اختفاء بعض الحروف أو استبدالها برموز غير مفهومة

إرشادات مهمة:
- تجاهل تماماً أي ملخص أو مستخلص موجود في المحتوى المقدم (عادةً ما يكون في بداية البحث). لم نقم بحذفه لأنه من الصعب حذف الملخص الموجود مسبقاً من النص المدخل، إلا أن مهمتك هي تجاهله بشكل كامل وإن

In [16]:
generate_from_title_and_content_prompts = []
for abstract in abstracts:
    paper_content = ' '.join(abstract['paper_extracted_content'].split(' ')[:max_paper_content_length])
    prompt = generate_from_title_and_content_prompt.format(title=abstract['title'], content=paper_content, max_paper_content_length=max_paper_content_length)
    generate_from_title_and_content_prompts.append(prompt)

In [17]:
generate_from_title_and_content_prompts[:3]

['أنت مساعد أكاديمي خبير متخصص في تحليل وإعادة صياغة النصوص العربية. مهمتك هي تحليل العنوان والمحتوى المقدم لك (بحد أقصى 400 كلمة) وإنشاء ملخص موجز وفعال من 100-150 كلمة.\n\nمهمتك الأساسية هي إنشاء ملخص أكاديمي (100-150 كلمة) في جميع الحالات، حتى لو كان النص المقدم غير مفهوم تماماً.\n\nالقاعدة الذهبية: يجب عليك دائماً إنشاء ملخص أكاديمي محترف، حتى لو كان عليك الاعتماد على العنوان فقط.\n\nملاحظات مهمة:\nالمحتوى المقدم مستخرج من ملف PDF وقد يحتوي على أخطاء كثيرة في التحويل\nقد تجد بعض المشاكل الشائعة مثل:\n- تداخل الحروف العربية بشكل كبير\n- تكرار بعض الحروف بشكل غير منطقي (مثل: السسسلام)\n- تشويه كبير في بعض الكلمات لدرجة قد تجعلها غير مفهومة\n- فصل غير صحيح للكلمات\n- أخطاء في علامات التشكيل\n- تحويل خاطئ لبعض الحروف المتشابهة مثل (ا/أ/إ) و (ه/ة)\n- اختفاء بعض الحروف أو استبدالها برموز غير مفهومة\n\nإرشادات مهمة:\n- تجاهل تماماً أي ملخص أو مستخلص موجود في المحتوى المقدم (عادةً ما يكون في بداية البحث). لم نقم بحذفه لأنه من الصعب حذف الملخص الموجود مسبقاً من النص المدخل، إلا أن مهمتك هي 

In [19]:
generate_abstracts(generation_prompts=generate_from_title_and_content_prompts, generation_type='from_title_and_content', batch_size=10)

Resuming from batch 0...


Processing batches:   0%|          | 0/300 [00:00<?, ?it/s]

Meta-Llama-3.1-70B-Instruct - batches sizes = [4, 4, 2]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 3/3 [03:04<00:00, 61.52s/it]


Meta-Llama-3.1-70B-Instruct - batches sizes = [3, 4, 3]


...

Meta-Llama-3.1-70B-Instruct - batches sizes = [1, 3, 4, 2]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 4/4 [05:54<00:00, 88.59s/it]


Meta-Llama-3.1-70B-Instruct - batches sizes = [4, 4, 2]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 3/3 [05:09<00:00, 103.23s/it]

All batches processed and saved to generated_arabic_datasets/llama-batched/arabic_abstracts_dataset/from_title_and_content_abstracts_generation.jsonl


## Generate by polishing abstracts

In [20]:
generate_by_polishing_abstract_prompt = """
أنت كاتب أكاديمي خبير. مهمتك هي تحسين وصقل الملخص المقدم مع مراعاة عنوان البحث.

المطلوب إنشاء ملخص موجز وفعال من فقرة واحدة متماسكة، دون فواصل سطرية أو حروف غير ضرورية. يجب أن يتراوح الملخص بين 100-150 كلمة وأن يتناسق بشكل جيد مع العنوان.

مهم: قم بإنشاء الملخص المحسّن فقط. ابدأ مباشرة بعد علامة <START> وانتهِ بعلامة <END>. لا تضِف أي كلمات قبل <START> أو بعد <END>. لا تضِف أي عبارات تمهيدية أو جمل ختامية أو أي نص ليس جزءاً من محتوى الملخص الأساسي.

إرشادات التحسين:

- تحسين الصياغة اللغوية والأسلوب الأكاديمي
- التأكد من التسلسل المنطقي للأفكار
- تعزيز الترابط بين العنوان والملخص
- الحفاظ على المعلومات الأساسية مع تحسين طريقة عرضها
- استخدام المصطلحات الأكاديمية المناسبة

العنوان:
{title}

الملخص الأصلي:
{abstract}

الملخص المحسّن:
<START>
""".strip()

In [21]:
print(generate_by_polishing_abstract_prompt.format(title=abstracts[0]['title'],abstract=abstracts[0]['arabic_abstract']))

أنت كاتب أكاديمي خبير. مهمتك هي تحسين وصقل الملخص المقدم مع مراعاة عنوان البحث.

المطلوب إنشاء ملخص موجز وفعال من فقرة واحدة متماسكة، دون فواصل سطرية أو حروف غير ضرورية. يجب أن يتراوح الملخص بين 100-150 كلمة وأن يتناسق بشكل جيد مع العنوان.

مهم: قم بإنشاء الملخص المحسّن فقط. ابدأ مباشرة بعد علامة <START> وانتهِ بعلامة <END>. لا تضِف أي كلمات قبل <START> أو بعد <END>. لا تضِف أي عبارات تمهيدية أو جمل ختامية أو أي نص ليس جزءاً من محتوى الملخص الأساسي.

إرشادات التحسين:

- تحسين الصياغة اللغوية والأسلوب الأكاديمي
- التأكد من التسلسل المنطقي للأفكار
- تعزيز الترابط بين العنوان والملخص
- الحفاظ على المعلومات الأساسية مع تحسين طريقة عرضها
- استخدام المصطلحات الأكاديمية المناسبة

العنوان:
صور عن نظام التعليم عند المرأة الأندلسية

الملخص الأصلي:
كثيرا ما ارتبطت المصادر التاريخية في الأندلس خاصة منها كتب التراجم والفهرسات والبرامج وغيرها بدراسة حياة العلماء والرواة والقضاة والساسة ؛ وقد تطورت هذه المادة حتى ترك لنا المؤلفون الأندلسيون سلسلة متواصلة الحلقات من كتب التـراجم كالصلة لابن بشكوال ، و

In [22]:
generate_by_polishing_abstract_prompts = []
for abstract in abstracts:
    prompt = generate_by_polishing_abstract_prompt.format(title=abstract['title'], abstract=abstract['arabic_abstract'])
    generate_by_polishing_abstract_prompts.append(prompt)

In [23]:
len(generate_by_polishing_abstract_prompts)

3000

In [24]:
generate_by_polishing_abstract_prompts[:3]

['أنت كاتب أكاديمي خبير. مهمتك هي تحسين وصقل الملخص المقدم مع مراعاة عنوان البحث.\n\nالمطلوب إنشاء ملخص موجز وفعال من فقرة واحدة متماسكة، دون فواصل سطرية أو حروف غير ضرورية. يجب أن يتراوح الملخص بين 100-150 كلمة وأن يتناسق بشكل جيد مع العنوان.\n\nمهم: قم بإنشاء الملخص المحسّن فقط. ابدأ مباشرة بعد علامة <START> وانتهِ بعلامة <END>. لا تضِف أي كلمات قبل <START> أو بعد <END>. لا تضِف أي عبارات تمهيدية أو جمل ختامية أو أي نص ليس جزءاً من محتوى الملخص الأساسي.\n\nإرشادات التحسين:\n\n- تحسين الصياغة اللغوية والأسلوب الأكاديمي\n- التأكد من التسلسل المنطقي للأفكار\n- تعزيز الترابط بين العنوان والملخص\n- الحفاظ على المعلومات الأساسية مع تحسين طريقة عرضها\n- استخدام المصطلحات الأكاديمية المناسبة\n\nالعنوان:\nصور عن نظام التعليم عند المرأة الأندلسية\n\nالملخص الأصلي:\nكثيرا ما ارتبطت المصادر التاريخية في الأندلس خاصة منها كتب التراجم والفهرسات والبرامج وغيرها بدراسة حياة العلماء والرواة والقضاة والساسة ؛ وقد تطورت هذه المادة حتى ترك لنا المؤلفون الأندلسيون سلسلة متواصلة الحلقات من كتب التـراجم كا

In [25]:
generate_abstracts(generation_prompts=generate_by_polishing_abstract_prompts, generation_type='by_polishing_abstracts')

Resuming from batch 0...


Processing batches:   0%|          | 0/30 [00:00<?, ?it/s]

Meta-Llama-3.1-70B-Instruct - batches sizes = [8, 10, 10, 10, 11, 11, 11, 11, 11, 7]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 10/10 [09:31<00:00, 57.12s/it]


Meta-Llama-3.1-70B-Instruct - batches sizes = [9, 10, 10, 10, 11, 11, 11, 11, 11, 6]


...

Meta-Llama-3.1-70B-Instruct - batches sizes = [8, 9, 10, 10, 11, 11, 11, 11, 11, 8]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 10/10 [09:20<00:00, 56.03s/it]


Meta-Llama-3.1-70B-Instruct - batches sizes = [8, 9, 10, 10, 10, 11, 11, 11, 11, 9]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 10/10 [09:02<00:00, 54.28s/it]

All batches processed and saved to generated_arabic_datasets/llama-batched/arabic_abstracts_dataset/by_polishing_abstracts_abstracts_generation.jsonl
